# Keras Overview

## Import tf.keras

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

## Build a simple model

In [4]:
model = tf.keras.Sequential()
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))

In [8]:
layers.Dense(64, activation='relu')
layers.Dense(64, activation=tf.nn.relu)
layers.Dense(64, kernel_regularizer=tf.keras.regularizers.l1(0.01))
layers.Dense(64, bias_regularizer=tf.keras.regularizers.l2(0.01))
layers.Dense(64, kernel_initializer='orthogonal')
layers.Dense(64, bias_initializer=tf.keras.initializers.Constant(2.0))

## Train and evaluate

In [11]:
model = tf.keras.Sequential([
    layers.Dense(64, activation=tf.nn.relu, input_shape=(32, )),
    layers.Dense(64, activation=tf.nn.relu),
    layers.Dense(10)
])

model.compile(optimizer=tf.keras.optimizers.Adam(0.1),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.Accuracy()])

In [24]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(0.01),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.Accuracy()])

In [25]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.1),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=[tf.keras.metrics.MeanSquaredError()])

In [26]:
import numpy as np

In [27]:
data = np.random.random((1000, 32))
data.shape

(1000, 32)

In [28]:
labels = np.random.random((1000, 10))
labels.shape

(1000, 10)

In [60]:
model.fit(data, labels, epochs=5, batch_size=32)

Epoch 1/5
1000/1000 [==============================] - 0s 130us/sample - loss: 0.3852 - mean_squared_error: 0.3852
Epoch 2/5
1000/1000 [==============================] - 0s 29us/sample - loss: 0.3595 - mean_squared_error: 0.3595
Epoch 3/5
1000/1000 [==============================] - 0s 31us/sample - loss: 0.3403 - mean_squared_error: 0.3403
Epoch 4/5
1000/1000 [==============================] - 0s 27us/sample - loss: 0.3132 - mean_squared_error: 0.3132
Epoch 5/5
1000/1000 [==============================] - 0s 26us/sample - loss: 0.2941 - mean_squared_error: 0.2941


In [31]:
val_data = np.random.random((100, 32))
val_data.shape

(100, 32)

In [32]:
val_labels = np.random.random((100, 10))
val_labels.shape

(100, 10)

In [61]:
model.fit(data,
          labels,
          epochs=5,
          batch_size=32,
          validation_data=(val_data, val_labels))

Train on 1000 samples, validate on 100 samples
Epoch 1/5
1000/1000 [==============================] - 0s 107us/sample - loss: 0.2783 - mean_squared_error: 0.2783 - val_loss: 0.2169 - val_mean_squared_error: 0.2169
Epoch 2/5
1000/1000 [==============================] - 0s 38us/sample - loss: 0.2531 - mean_squared_error: 0.2531 - val_loss: 0.2623 - val_mean_squared_error: 0.2623
Epoch 3/5
1000/1000 [==============================] - 0s 38us/sample - loss: 0.2467 - mean_squared_error: 0.2467 - val_loss: 0.2104 - val_mean_squared_error: 0.2104
Epoch 4/5
1000/1000 [==============================] - 0s 38us/sample - loss: 0.2172 - mean_squared_error: 0.2172 - val_loss: 0.3457 - val_mean_squared_error: 0.3457
Epoch 5/5
1000/1000 [==============================] - 0s 32us/sample - loss: 0.2142 - mean_squared_error: 0.2142 - val_loss: 0.1737 - val_mean_squared_error: 0.1737


In [36]:
model.evaluate(data, labels, batch_size=32)

1000/1000 [==============================] - 0s 20us/sample - loss: 0.1549 - mean_squared_error: 0.1549


[0.15491807007789613, 0.15491806]

In [37]:
dataset = tf.data.Dataset.from_tensor_slices((data, labels))
dataset = dataset.batch(32)

model.evaluate(dataset)

32/32 [==============================] - 0s 2ms/step - loss: 0.1554 - mean_squared_error: 0.1549


[0.15538878180086613, 0.15491806]

In [40]:
result = model.predict(dataset)
result.shape

(1000, 10)

## Build complex models

In [42]:
inputs = tf.keras.Input(shape=(32, ))
x = layers.Dense(64, activation=tf.nn.relu)(inputs)
x = layers.Dense(64, activation=tf.nn.relu)(x)
predictions = layers.Dense(10)(x)

In [193]:
model = tf.keras.Model(inputs=inputs, outputs=predictions)

model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=[tf.keras.metrics.MeanSquaredError()])

In [59]:
dataset = dataset.shuffle(1000, reshuffle_each_iteration=True)
model.fit(dataset, epochs=5)

Epoch 1/5
32/32 [==============================] - 0s 4ms/step - loss: 0.5406 - mean_squared_error: 0.5403
Epoch 2/5
32/32 [==============================] - 0s 1ms/step - loss: 0.5110 - mean_squared_error: 0.5079
Epoch 3/5
32/32 [==============================] - 0s 855us/step - loss: 0.4817 - mean_squared_error: 0.4818
Epoch 4/5
32/32 [==============================] - 0s 850us/step - loss: 0.4624 - mean_squared_error: 0.4635
Epoch 5/5
32/32 [==============================] - 0s 834us/step - loss: 0.4152 - mean_squared_error: 0.4110


In [71]:
class MyModel(tf.keras.Model):
    def __init__(self, num_classes=10):
        super(MyModel, self).__init__(name='my_model')
        self.num_classes = num_classes
        self.dense_1 = layers.Dense(32, activation=tf.nn.relu)
        self.dense_2 = layers.Dense(num_classes)

    # Call on fit method
    def call(self, inputs):
        x = self.dense_1(inputs)
        return self.dense_2(x)

In [72]:
model = MyModel(num_classes=10)

In [73]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.Accuracy()])

In [74]:
model.fit(dataset, epochs=5)

Epoch 1/5
32/32 [==============================] - 0s 5ms/step - loss: 12.1425 - accuracy: 0.0000e+00
Epoch 2/5
32/32 [==============================] - 0s 763us/step - loss: 13.9697 - accuracy: 0.0000e+00
Epoch 3/5
32/32 [==============================] - 0s 844us/step - loss: 15.9812 - accuracy: 0.0000e+00
Epoch 4/5
32/32 [==============================] - 0s 820us/step - loss: 17.7008 - accuracy: 0.0000e+00
Epoch 5/5
32/32 [==============================] - 0s 671us/step - loss: 19.1393 - accuracy: 0.0000e+00


In [118]:
class MyLayer(layers.Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(MyLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.kernel = self.add_weight(name='kernel',
                                      shape=(input_shape[1], self.output_dim),
                                      initializer='uniform',
                                      trainable=True)

    def call(self, inputs):
        return tf.matmul(inputs, self.kernel)

    def get_config(self):
        base_config = super(MyLayer, self).get_config()
        base_config['output_dim'] = self.output_dim
        return base_config

    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [119]:
model = tf.keras.Sequential([MyLayer(10)])

In [120]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [121]:
model.fit(data, labels, batch_size=32, epochs=5)

TypeError: Failed to convert object of type <class 'tuple'> to Tensor. Contents: (Dimension(32), 10). Consider casting elements to a supported type.

## Callbacks

In [132]:
model = MyModel(num_classes=10)

In [133]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.Accuracy()])

In [135]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=5, monitor='val_loss'),
    tf.keras.callbacks.TensorBoard(log_dir='./logs')
]

In [137]:
model.fit(data,
          labels,
          epochs=5,
          callbacks=callbacks,
          validation_data=(val_data, val_labels))

Train on 1000 samples, validate on 100 samples
Epoch 1/5
1000/1000 [==============================] - 0s 33us/sample - loss: 24.1635 - accuracy: 0.0000e+00 - val_loss: 25.2670 - val_accuracy: 0.0000e+00
Epoch 2/5
1000/1000 [==============================] - 0s 36us/sample - loss: 25.8069 - accuracy: 0.0000e+00 - val_loss: 26.7220 - val_accuracy: 0.0000e+00
Epoch 3/5
1000/1000 [==============================] - 0s 34us/sample - loss: 27.2806 - accuracy: 0.0000e+00 - val_loss: 28.6135 - val_accuracy: 0.0000e+00
Epoch 4/5
1000/1000 [==============================] - 0s 30us/sample - loss: 28.9105 - accuracy: 0.0000e+00 - val_loss: 30.0936 - val_accuracy: 0.0000e+00
Epoch 5/5
1000/1000 [==============================] - 0s 29us/sample - loss: 30.4319 - accuracy: 0.0000e+00 - val_loss: 31.3043 - val_accuracy: 0.0000e+00


## Save and restore

In [138]:
model.save_weights('./weights/my_model')

In [142]:
model.load_weights('./weights/my_model')

In [150]:
inputs = tf.keras.Input(shape=(32, ))
x = layers.Dense(64, activation=tf.nn.relu)(inputs)
x = layers.Dense(64, activation=tf.nn.relu)(x)
predictions = layers.Dense(10)(x)
model = tf.keras.Model(inputs=inputs, outputs=predictions)
json_string = model.to_json()

In [151]:
import json
import pprint
pprint.pprint(json.loads(model.to_json()))

{'backend': 'tensorflow',
 'class_name': 'Model',
 'config': {'input_layers': [['input_3', 0, 0]],
            'layers': [{'class_name': 'InputLayer',
                        'config': {'batch_input_shape': [None, 32],
                                   'dtype': 'float32',
                                   'name': 'input_3',
                                   'sparse': False},
                        'inbound_nodes': [],
                        'name': 'input_3'},
                       {'class_name': 'Dense',
                        'config': {'activation': 'relu',
                                   'activity_regularizer': None,
                                   'bias_constraint': None,
                                   'bias_initializer': {'class_name': 'Zeros',
                                                        'config': {'dtype': 'float32'}},
                                   'bias_regularizer': None,
                                   'dtype': 'float32',
                 

In [152]:
tf.keras.models.model_from_json(json_string)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [153]:
model.save('my_model')

In [155]:
model = tf.keras.models.load_model('my_model')

## Distribution

In [194]:
strategy = tf.distribute.MirroredStrategy()

In [195]:
with strategy.scope():
    model = tf.keras.Sequential()
    model.add(layers.Dense(16, activation='relu', input_shape=(10, )))
    model.add(layers.Dense(1))

    model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                  optimizer=tf.keras.optimizers.Adam(0.01),
                  metrics=[tf.keras.metrics.Accuracy()])

    model.summary()

Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_75 (Dense)             (None, 16)                176       
_________________________________________________________________
dense_76 (Dense)             (None, 1)                 17        
Total params: 193
Trainable params: 193
Non-trainable params: 0
_________________________________________________________________


In [196]:
import time

In [197]:
x = np.random.random((150000, 10))
y = np.random.randint(2, size=(150000, 1))
x = tf.cast(x, tf.float32)
dataset = tf.data.Dataset.from_tensor_slices((x, y))
dataset = dataset.shuffle(buffer_size=150000).batch(32)

start = time.time()
model.fit(dataset, epochs=100)
end = time.time()

Epoch 1/100
4688/4688 [==============================] - 5s 965us/step - loss: 0.6935 - accuracy: 0.0000e+00
Epoch 2/100
4688/4688 [==============================] - 4s 849us/step - loss: 0.6933 - accuracy: 0.0000e+00
Epoch 3/100
4688/4688 [==============================] - 4s 848us/step - loss: 0.6933 - accuracy: 0.0000e+00
Epoch 4/100
4688/4688 [==============================] - 4s 846us/step - loss: 0.6934 - accuracy: 0.0000e+00
Epoch 5/100
4688/4688 [==============================] - 4s 849us/step - loss: 0.6934 - accuracy: 0.0000e+00
Epoch 6/100
4688/4688 [==============================] - 4s 898us/step - loss: 0.6934 - accuracy: 0.0000e+00
Epoch 7/100
4688/4688 [==============================] - 4s 851us/step - loss: 0.6934 - accuracy: 0.0000e+00
Epoch 8/100
4688/4688 [==============================] - 4s 841us/step - loss: 0.6934 - accuracy: 0.0000e+00
Epoch 9/100
4688/4688 [==============================] - 4s 858us/step - loss: 0.6934 - accuracy: 0.0000e+00
Epoch 10/100
4688/4

4688/4688 [==============================] - 5s 969us/step - loss: 0.6933 - accuracy: 0.0000e+00
Epoch 76/100
4688/4688 [==============================] - 4s 872us/step - loss: 0.6933 - accuracy: 0.0000e+00
Epoch 77/100
4688/4688 [==============================] - 4s 879us/step - loss: 0.6933 - accuracy: 0.0000e+00
Epoch 78/100
4688/4688 [==============================] - 4s 854us/step - loss: 0.6934 - accuracy: 0.0000e+00
Epoch 79/100
4688/4688 [==============================] - 4s 851us/step - loss: 0.6934 - accuracy: 0.0000e+00
Epoch 80/100
4688/4688 [==============================] - 4s 849us/step - loss: 0.6933 - accuracy: 0.0000e+00
Epoch 81/100
4688/4688 [==============================] - 4s 880us/step - loss: 0.6934 - accuracy: 0.0000e+00
Epoch 82/100
4688/4688 [==============================] - 4s 916us/step - loss: 0.6934 - accuracy: 0.0000e+00
Epoch 83/100
4688/4688 [==============================] - 5s 983us/step - loss: 0.6933 - accuracy: 0.0000e+00
Epoch 84/100
4688/4688 

In [198]:
end - start

485.8176860809326

In [199]:
model = tf.keras.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10, )))
model.add(layers.Dense(1))

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(0.01),
              metrics=[tf.keras.metrics.Accuracy()])

model.summary()

Model: "sequential_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_77 (Dense)             (None, 16)                176       
_________________________________________________________________
dense_78 (Dense)             (None, 1)                 17        
Total params: 193
Trainable params: 193
Non-trainable params: 0
_________________________________________________________________


In [200]:
x = np.random.random((150000, 10))
y = np.random.randint(2, size=(150000, 1))
x = tf.cast(x, tf.float32)
dataset = tf.data.Dataset.from_tensor_slices((x, y))
dataset = dataset.shuffle(buffer_size=150000).batch(32)

start = time.time()
model.fit(dataset, epochs=100)
end = time.time()

Epoch 1/100
4688/4688 [==============================] - 5s 983us/step - loss: 0.6935 - accuracy: 0.0000e+00
Epoch 2/100
4688/4688 [==============================] - 4s 888us/step - loss: 0.6934 - accuracy: 0.0000e+00
Epoch 3/100
4688/4688 [==============================] - 4s 892us/step - loss: 0.6933 - accuracy: 0.0000e+00
Epoch 4/100
4688/4688 [==============================] - 4s 891us/step - loss: 0.6934 - accuracy: 0.0000e+00
Epoch 5/100
4688/4688 [==============================] - 4s 901us/step - loss: 0.6934 - accuracy: 0.0000e+00
Epoch 6/100
4688/4688 [==============================] - 4s 894us/step - loss: 0.6934 - accuracy: 0.0000e+00
Epoch 7/100
4688/4688 [==============================] - 4s 888us/step - loss: 0.6934 - accuracy: 0.0000e+00
Epoch 8/100
4688/4688 [==============================] - 4s 888us/step - loss: 0.6934 - accuracy: 0.0000e+00
Epoch 9/100
4688/4688 [==============================] - 4s 892us/step - loss: 0.6934 - accuracy: 0.0000e+00
Epoch 10/100
4688/4

4688/4688 [==============================] - 5s 988us/step - loss: 0.6934 - accuracy: 0.0000e+00
Epoch 76/100
4688/4688 [==============================] - 4s 920us/step - loss: 0.6933 - accuracy: 0.0000e+00
Epoch 77/100
4688/4688 [==============================] - 4s 901us/step - loss: 0.6934 - accuracy: 0.0000e+00
Epoch 78/100
4688/4688 [==============================] - 4s 892us/step - loss: 0.6933 - accuracy: 0.0000e+00
Epoch 79/100
4688/4688 [==============================] - 4s 883us/step - loss: 0.6933 - accuracy: 0.0000e+00
Epoch 80/100
4688/4688 [==============================] - 4s 887us/step - loss: 0.6934 - accuracy: 0.0000e+00
Epoch 81/100
4688/4688 [==============================] - 4s 916us/step - loss: 0.6933 - accuracy: 0.0000e+00
Epoch 82/100
4688/4688 [==============================] - 4s 928us/step - loss: 0.6934 - accuracy: 0.0000e+00
Epoch 83/100
4688/4688 [==============================] - 4s 945us/step - loss: 0.6934 - accuracy: 0.0000e+00
Epoch 84/100
4688/4688 

In [201]:
end - start

431.5237398147583